In [5]:
import pandas as pd

La magia de matplotlib notebook, es muy útil para hacer plots más visibles, pero es un poco complicada de usa, ya que require cerrar cada plot para poder continuar.

Alternativamente, se puede usar la magia de %matplotlib inline, que es más sencilla, ya que simplemente muestra el gráfico original en jupyter notebook. Para modificar el tamaño de los plots en este caso, basta con cambiar el parámetro general de matplotlib figure.figsize al tamaño de gráfico deseado (en pulgadas).

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10,10) 

En este apartado vamoas a revisar el dataset. El objetibvo será tener un dataset lo más parecido al original, pero sin fallos en el mismo que puedan llevarnos a sacar conclusiones erróneas.

El input de ete paso es el output del anterior.

In [7]:
vehiculos = pd.read_csv("./vehiculos.1.procesado_inicial.csv")

In [8]:
vehiculos.shape

(38436, 11)

#### Duplicados
En este apartado se intentan ver dos cosas:

- Asegurarse de que los datos que no deberían estar duplicados no lo están (ejemplo, números de teléfono)
- Revisar si hay columnas que tengan un alto número de duplicados, ya que no aportan mucha información

Recordemos que las entidades posibles eran:

* fabricante
* fabricante-modelo
* fabricante-modelo-año
* fabricante-año

De estas entidades, las únicas que no se deberían repetir coches específicos, es decir, fabricante-modelo-año


In [9]:
vehiculos["modelo_unico"] = vehiculos.fabricante.str.cat([vehiculos.modelo, vehiculos.year.apply(str)], sep="-")

In [10]:
vehiculos.modelo_unico.value_counts()

Jeep-Cherokee/Wagoneer-1985               24
Ford-F150 Pickup 2WD-1984                 19
GMC-C15 Pickup 2WD-1984                   19
Chevrolet-C10 Pickup 2WD-1984             19
GMC-C15 Pickup 2WD-1985                   18
                                          ..
Mercedes-Benz-C300-2015                    1
Mercedes-Benz-C300-2009                    1
Mercedes-Benz-C280 4matic-2007             1
Mercedes-Benz-C280 4matic-2006             1
smart-fortwo electric drive coupe-2016     1
Name: modelo_unico, Length: 17448, dtype: int64

Nos damos cuenta de que hay muchos repetidos.

In [11]:
vehiculos[vehiculos.modelo_unico=="Chevrolet-C1500 Pickup 2WD-1991"].head()

,fabricante,modelo,year,desplazamiento,cilindros,trany,traccion,clase,combustible,consumo,co2,modelo_unico
4957,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Automatic 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4958,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Manual 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4959,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Manual 5-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4960,Chevrolet,C1500 Pickup 2WD,1991,4.3,6.0,Manual 5-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,17,522.764706,Chevrolet-C1500 Pickup 2WD-1991
4961,Chevrolet,C1500 Pickup 2WD,1991,5.0,8.0,Automatic 4-spd,Rear-Wheel Drive,Standard Pickup Trucks,Regular,15,592.466667,Chevrolet-C1500 Pickup 2WD-1991


Como vemos, cada modelo único tiene diferentes configuraciones de cada coche (por ejemplo, 3 o 4 velocidades)
Por lo tanto, solo vamos a considerar duplicados aquellos que sean idénticos a todas sus columnas

In [12]:
vehiculos[vehiculos.duplicated()].shape

(1506, 12)

Vemos que hay 1506 duplicados, podemos eliminarlos para el resto del análisis, ya que pueden distorsionar las conclusiones

In [13]:
vehiculos = vehiculos.drop_duplicates()
vehiculos.shape

(36930, 12)

Borramos la columna modelo_unico

In [14]:
del vehiculos["modelo_unico"]

Ahora falta ver si hay variables en las que haya una gran cantidad de records que tengan el mismo valor (cardinalidad)

In [16]:
n_records = len(vehiculos)
def valores_duplicados_col(df):
    for columna in df:
        n_por_valor = df[columna].value_counts()
        mas_comun = n_por_valor.iloc[0]
        menos_comun = n_por_valor.iloc[-1]
        print("{} | {}-{} | {}".format(
            df[columna].name,
            round(mas_comun / (1.0 * n_records), 3),
            round(menos_comun / (1.0 * n_records), 3),
            df[columna].dtype
        ))
        
valores_duplicados_col(vehiculos)

fabricante | 0.1-0.0 | object
modelo | 0.005-0.0 | object
year | 0.038-0.007 | int64
desplazamiento | 0.095-0.0 | float64
cilindros | 0.38-0.0 | float64
trany | 0.287-0.0 | object
traccion | 0.353-0.005 | object
clase | 0.145-0.0 | object
combustible | 0.652-0.0 | object
consumo | 0.097-0.0 | int64
co2 | 0.084-0.0 | float64


Vemos que los campos traccion, transmission, cilindros y combustible pueden tener un problema de valores repetidos, en realidad, 30% no es significativo, sería distinto si el valor más común tuviera un 90%

In [18]:
vehiculos.traccion.value_counts(normalize=True)

Front-Wheel Drive             0.360280
Rear-Wheel Drive              0.352863
4-Wheel or All-Wheel Drive    0.177345
All-Wheel Drive               0.062325
4-Wheel Drive                 0.030886
2-Wheel Drive                 0.011402
Part-time 4-Wheel Drive       0.004899
Name: traccion, dtype: float64